# Minggu 5: Sinyal ECG & Deteksi QRS
## *ECG Signals & QRS Detection*

**Mata Kuliah:** Pengolahan Sinyal Medis -- Universitas Indonesia

---

### Tujuan Pembelajaran

Setelah menyelesaikan demo ini, mahasiswa diharapkan mampu:

1. Memahami sistem konduksi listrik jantung dan asal-usul sinyal ECG.
2. Mengenali komponen **kompleks PQRST** beserta nilai normalnya dan relevansi klinisnya.
3. Melakukan **preprocessing ECG** secara bertahap: notch filter, baseline removal, bandpass filter.
4. Memahami dan mengimplementasikan prinsip **algoritma Pan-Tompkins** untuk deteksi QRS.
5. Mengevaluasi kinerja detektor R-peak menggunakan metrik Sensitivity dan PPV.
6. Menghitung dan menginterpretasikan **interval RR** dan **Heart Rate Variability (HRV)**.

---
## 1. Pendahuluan: Sistem Konduksi Jantung

**ECG** (*Electrocardiogram*) adalah rekaman aktivitas listrik jantung yang diukur dari permukaan kulit. ECG merupakan alat diagnostik kardiovaskular yang paling umum digunakan di dunia karena sifatnya yang **non-invasif**, murah, dan informatif.

### Sistem Konduksi Listrik Jantung (*Cardiac Electrical Conduction System*)

Setiap detak jantung diawali oleh impuls listrik yang merambat melalui jalur konduksi yang terstruktur:

| Tahap | Struktur | Fungsi |
|-------|----------|--------|
| 1 | **SA Node** (*Sinoatrial Node*) | Pacemaker alami; menghasilkan impuls 60-100x/menit |
| 2 | **AV Node** (*Atrioventricular Node*) | Menunda konduksi ~0.1 s; memungkinkan pengisian ventrikel |
| 3 | **Bundle of His** | Menghantarkan impuls dari AV node ke septum |
| 4 | **Berkas Cabang** (*Bundle Branches*) | Kiri & kanan; menuju ventrikel kiri & kanan |
| 5 | **Serat Purkinje** (*Purkinje Fibers*) | Mendistribusikan impuls ke seluruh miokardium ventrikel |

### Kompleks PQRST dan Nilai Normal

Setiap siklus detak jantung menghasilkan pola gelombang yang disebut **kompleks PQRST**:

| Komponen | Durasi Normal | Makna Fisiologis |
|----------|--------------|------------------|
| **Gelombang P** | 80--100 ms | Depolarisasi atrium (kontraksi serambi) |
| **Interval PR** | 120--200 ms | Waktu konduksi dari SA node ke ventrikel |
| **Kompleks QRS** | 60--100 ms | Depolarisasi ventrikel (kontraksi bilik) |
| **Interval QT** | 360--440 ms | Siklus listrik ventrikel lengkap |
| **Gelombang T** | 160 ms | Repolarisasi ventrikel (pemulihan bilik) |
| **Interval RR** | 600--1000 ms | Satu siklus detak jantung penuh |

### Relevansi Klinis Gangguan ECG

| Kondisi | Temuan ECG | Mekanisme |
|---------|-----------|----------|
| **Blok AV derajat 1** | PR interval > 200 ms | Konduksi AV node lambat |
| **Blok Cabang Berkas** (*Bundle Branch Block*) | QRS > 120 ms, pola rSR' | Konduksi ventrikel terganggu |
| **Long QT Syndrome** | QT interval > 440 ms | Risiko Torsades de Pointes |
| **Fibrilasi Atrium** | Tidak ada gelombang P, irama RR ireguler | Depolarisasi atrium kacau |
| **Infark Miokard Akut** | Elevasi segmen ST | Iskemia miokardium |
| **Bradikardia** | Heart rate < 60 bpm | SA node lambat atau blok konduksi |
| **Takikardia** | Heart rate > 100 bpm | Peningkatan automatisitas atau re-entry |

### Hubungan Heart Rate dan Interval RR

$$\text{Heart Rate (bpm)} = \frac{60}{\text{RR interval (s)}}$$

Contoh: Heart rate 75 bpm $\Rightarrow$ RR interval = $60/75 = 0.8$ s. Heart rate 60 bpm $\Rightarrow$ RR interval = 1.0 s.

In [ ]:
%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt
from scipy import signal as scipy_signal

# Library kursus
from medsinyal.io import load_synthetic
from medsinyal.viz import plot_signal, plot_ecg
from medsinyal.filters import (
    bandpass_filter, notch_filter, remove_baseline_wander, moving_average
)
from medsinyal.ecg import (
    preprocess_ecg,
    detect_r_peaks,
    compute_rr_intervals,
    compute_heart_rate,
    compute_hrv_features,
    extract_ecg_features,
)

# Konfigurasi plot global
plt.rcParams['figure.figsize'] = (14, 4)
plt.rcParams['figure.dpi'] = 100
plt.rcParams['axes.grid'] = True

print('Semua library berhasil dimuat.')

---
## 2. Karakteristik Sinyal ECG (*ECG Signal Characteristics*)

Sebelum memproses sinyal, kita perlu memahami karakteristik dasarnya -- amplitudo, durasi, dan kandungan frekuensinya.

### Kandungan Frekuensi Sinyal ECG

| Komponen | Rentang Frekuensi | Keterangan |
|----------|------------------|------------|
| **Baseline wander** | < 0.5 Hz | Drift akibat pernapasan dan gerakan elektroda |
| **Gelombang P dan T** | 0.5 -- 10 Hz | Depolarisasi/repolarisasi lambat |
| **Kompleks QRS** | 5 -- 40 Hz | Aktivasi ventrikel cepat |
| **Interferensi jala listrik** | 50 Hz (Indonesia) | *Power line interference* |

Untuk ECG diagnostik, bandpass 0.05--150 Hz digunakan. Untuk monitoring, 0.5--40 Hz sudah cukup.

In [ ]:
# Memuat data ECG sintetis
data = load_synthetic('synthetic_ecg')

print('Keys dalam file:', list(data.keys()))
print(f'Sampling rate  : {float(data["fs"])} Hz')
print(f'Heart rate     : {float(data["heart_rate"])} bpm')
print(f'Durasi sinyal  : {data["t"][-1]:.1f} s')
print(f'Jumlah sampel  : {len(data["t"])}')
print(f'Jumlah R-peak  : {len(data["r_peak_indices"])}')

# Ekstrak variabel utama
t           = data['t']
fs          = float(data['fs'])
ecg_clean   = data['ecg_clean']
ecg_noisy   = data['ecg_noisy']
r_peaks_gt  = data['r_peak_indices']   # ground truth R-peak indices
hr_gt       = float(data['heart_rate'])  # ground truth heart rate (bpm)

# Interval RR ground truth
rr_gt_s = 60.0 / hr_gt
print(f'\nInterval RR    : {rr_gt_s:.3f} s  ({hr_gt:.1f} bpm)')

In [ ]:
# --- Tampilkan ECG bersih vs noisy ---
fig, axes = plt.subplots(2, 1, figsize=(14, 7), sharex=True)
fig.suptitle('Sinyal ECG Sintetis: Bersih vs Ternoise', fontsize=14, fontweight='bold')

plot_ecg(t, ecg_clean, r_peaks=r_peaks_gt,
         title='ECG Bersih (*Clean ECG*) dengan Ground-Truth R-peak', ax=axes[0])
axes[0].set_ylabel('Amplitudo (mV)')

plot_ecg(t, ecg_noisy,
         title='ECG Ternoise (*Noisy ECG*) -- Baseline Wander + Interferensi 50 Hz + Random Noise',
         ax=axes[1])
axes[1].set_ylabel('Amplitudo (mV)')

plt.tight_layout()
plt.show()

# Statistik dasar
print('Statistik ECG:')
for label, sig in [('ECG Bersih', ecg_clean), ('ECG Noisy', ecg_noisy)]:
    print(f'  {label:<15}: Mean={np.mean(sig):+.4f} mV, '
          f'Std={np.std(sig):.4f} mV, '
          f'Min={np.min(sig):.4f} mV, Max={np.max(sig):.4f} mV')

In [ ]:
# --- Zoom pada 3 detak jantung untuk melihat PQRST ---
# Pilih segmen yang mencakup 3 R-peak pertama
beat_start = max(0, r_peaks_gt[0] - int(0.3 * fs))
beat_end   = min(len(t), r_peaks_gt[3] + int(0.3 * fs))  # R-peak ke-4 sebagai batas
t_zoom     = t[beat_start:beat_end]
ecg_zoom   = ecg_clean[beat_start:beat_end]
# R-peak yang masuk ke dalam zoom window
rp_zoom    = r_peaks_gt[(r_peaks_gt >= beat_start) & (r_peaks_gt < beat_end)]

fig, ax = plt.subplots(figsize=(14, 5))
ax.plot(t_zoom, ecg_zoom, 'k-', linewidth=1.5)
ax.plot(t[rp_zoom], ecg_clean[rp_zoom], 'rv', markersize=10,
        label='R-peak', zorder=5)

# Anotasi PQRST pada beat kedua (indeks 1)
rp_idx = rp_zoom[1]  # indeks global dari R-peak kedua dalam zoom
rp_t   = t[rp_idx]
annotations = [
    ('P',  rp_t - 0.14, 0.15,  'blue'),
    ('Q',  rp_t - 0.03, -0.10, 'darkgreen'),
    ('R',  rp_t,         1.00,  'red'),
    ('S',  rp_t + 0.03, -0.20, 'purple'),
    ('T',  rp_t + 0.20,  0.30, 'darkorange'),
]
for lbl, x, y, c in annotations:
    ax.annotate(
        lbl, xy=(x, y), xytext=(x, y + 0.25),
        fontsize=16, fontweight='bold', color=c, ha='center',
        arrowprops=dict(arrowstyle='->', color=c, lw=1.5)
    )

ax.set_title('Zoom: 3 Detak Jantung -- Kompleks PQRST', fontsize=13)
ax.set_xlabel('Waktu (s)')
ax.set_ylabel('Amplitudo (mV)')
ax.legend(loc='upper right')
plt.tight_layout()
plt.show()

print('Gelombang P : Depolarisasi atrium (mV kecil, sebelum QRS)')
print('Kompleks QRS: Depolarisasi ventrikel (amplitudo terbesar)')
print('Gelombang T : Repolarisasi ventrikel (mV sedang, setelah QRS)')

In [ ]:
# --- Analisis spektrum frekuensi (FFT) ---
N   = len(ecg_clean)
fft_clean = np.abs(np.fft.rfft(ecg_clean)) / N
fft_noisy = np.abs(np.fft.rfft(ecg_noisy)) / N
freqs     = np.fft.rfftfreq(N, d=1.0 / fs)

fig, ax = plt.subplots(figsize=(14, 5))
ax.semilogy(freqs, fft_clean, 'b-', linewidth=0.9, alpha=0.85, label='ECG Bersih')
ax.semilogy(freqs, fft_noisy, 'r-', linewidth=0.9, alpha=0.7,  label='ECG Noisy')

# Anotasi rentang komponen frekuensi
ax.axvspan(0,    0.5,  alpha=0.08, color='gray',   label='Baseline wander (< 0.5 Hz)')
ax.axvspan(0.5,  10,   alpha=0.08, color='blue',   label='P/T waves (0.5-10 Hz)')
ax.axvspan(5,    40,   alpha=0.08, color='green',  label='QRS complex (5-40 Hz)')
ax.axvline(50,         color='red', linestyle='--', linewidth=1.5, label='Interferensi 50 Hz')

ax.set_xlim(0, 120)
ax.set_xlabel('Frekuensi (Hz)')
ax.set_ylabel('Amplitudo (skala log)')
ax.set_title('Spektrum Frekuensi ECG -- FFT', fontsize=13)
ax.legend(loc='upper right', fontsize=9)
plt.tight_layout()
plt.show()

# Perbandingan daya pada pita noise (50 Hz)
mask_50 = (freqs >= 48) & (freqs <= 52)
print(f'Daya pada 50 Hz (bersih): {np.sum(fft_clean[mask_50]**2):.6f}')
print(f'Daya pada 50 Hz (noisy) : {np.sum(fft_noisy[mask_50]**2):.6f}')
print('Peningkatan daya pada 50 Hz mengonfirmasi adanya interferensi jala listrik.')

---
## 3. Preprocessing ECG (*ECG Preprocessing*)

Sebelum analisis, sinyal ECG harus dibersihkan dari noise. Urutan preprocessing yang umum:

1. **Notch filter 50 Hz** -- Menghilangkan interferensi jala listrik (*power line interference*)
2. **Baseline wander removal** -- Menghilangkan drift frekuensi rendah (< 0.5 Hz) akibat pernapasan
3. **Bandpass filter 0.5--40 Hz** -- Mempertahankan hanya komponen ECG yang relevan

Kita akan menerapkan setiap langkah secara bertahap dan mengukur perbaikan menggunakan **RMSE** (*Root Mean Square Error*) terhadap sinyal bersih referensi:

$$\text{RMSE} = \sqrt{\frac{1}{N}\sum_{i=1}^{N}(x_i - x_{\text{ref},i})^2}$$

In [ ]:
# --- Preprocessing bertahap ---

# Langkah 1: Notch filter 50 Hz
ecg_step1 = notch_filter(ecg_noisy, freq=50.0, fs=fs)

# Langkah 2: Hapus baseline wander
ecg_step2 = remove_baseline_wander(ecg_step1, fs=fs)

# Langkah 3: Bandpass filter 0.5-40 Hz
ecg_step3 = bandpass_filter(ecg_step2, lowcut=0.5, highcut=40.0, fs=fs, order=4)

# Hitung RMSE di setiap tahap
def rmse(x, ref):
    return np.sqrt(np.mean((x - ref) ** 2))

rmse_raw   = rmse(ecg_noisy, ecg_clean)
rmse_step1 = rmse(ecg_step1, ecg_clean)
rmse_step2 = rmse(ecg_step2, ecg_clean)
rmse_step3 = rmse(ecg_step3, ecg_clean)

print('RMSE terhadap ECG bersih di setiap tahap preprocessing:')
print(f'  [0] ECG Noisy (raw)         : RMSE = {rmse_raw:.6f} mV')
print(f'  [1] Setelah Notch 50 Hz     : RMSE = {rmse_step1:.6f} mV  '
      f'({(1-rmse_step1/rmse_raw)*100:.1f}% perbaikan)')
print(f'  [2] Setelah Baseline Removal: RMSE = {rmse_step2:.6f} mV  '
      f'({(1-rmse_step2/rmse_raw)*100:.1f}% perbaikan)')
print(f'  [3] Setelah Bandpass 0.5-40 : RMSE = {rmse_step3:.6f} mV  '
      f'({(1-rmse_step3/rmse_raw)*100:.1f}% perbaikan)')

# Visualisasi 4 tahap (zoom pada 3 detik pertama)
t_show = 3.0
mask   = t <= t_show

fig, axes = plt.subplots(4, 1, figsize=(14, 14), sharex=True)
fig.suptitle('Tahapan Preprocessing ECG (*ECG Preprocessing Steps*)',
             fontsize=14, fontweight='bold')

steps = [
    (ecg_noisy, 'tab:red',    '[0] ECG Noisy (raw)'),
    (ecg_step1, 'tab:orange', '[1] Setelah Notch Filter 50 Hz'),
    (ecg_step2, 'tab:blue',   '[2] Setelah Baseline Wander Removal'),
    (ecg_step3, 'tab:green',  '[3] Setelah Bandpass Filter 0.5-40 Hz'),
]

for i, (sig, color, title) in enumerate(steps):
    axes[i].plot(t[mask], ecg_clean[mask], 'k-', linewidth=0.8,
                 alpha=0.4, label='Referensi (clean)')
    axes[i].plot(t[mask], sig[mask], color=color, linewidth=1.2, label=title)
    axes[i].set_ylabel('Amplitudo (mV)')
    axes[i].set_title(title, fontsize=11)
    axes[i].legend(loc='upper right', fontsize=9)

axes[-1].set_xlabel('Waktu (s)')
plt.tight_layout()
plt.show()

In [ ]:
# --- Gunakan preprocess_ecg() dari medsinyal untuk pipeline lengkap ---
ecg_preprocessed = preprocess_ecg(
    ecg_noisy, fs,
    remove_baseline=True,
    remove_powerline=True,
    bandpass=(0.5, 40.0)
)

rmse_preproc = rmse(ecg_preprocessed, ecg_clean)
print(f'RMSE setelah preprocess_ecg(): {rmse_preproc:.6f} mV '
      f'({(1-rmse_preproc/rmse_raw)*100:.1f}% perbaikan dari raw)')

# Simpan sebagai sinyal yang akan digunakan untuk analisis selanjutnya
ecg_proc = ecg_preprocessed.copy()
print('ECG terpreproses siap untuk deteksi QRS.')

---
## 4. Algoritma Pan-Tompkins untuk Deteksi QRS

**Algoritma Pan-Tompkins** (Pan & Tompkins, 1985) adalah standar emas (*gold standard*) untuk deteksi QRS kompleks secara *real-time*. Algoritma ini memproses sinyal melalui serangkaian tahap untuk menonjolkan QRS dan menekan P/T wave serta noise.

### Langkah-Langkah Algoritma

| Tahap | Operasi | Tujuan |
|-------|---------|--------|
| 1 | **Bandpass 5-15 Hz** | Fokus pada frekuensi QRS; tekan P, T, baseline |
| 2 | **Derivatif** (*Derivative*) | Deteksi kemiringan (*slope*) tajam QRS |
| 3 | **Kuadrat** (*Squaring*) | Perkuat QRS, buat semua nilai positif |
| 4 | **Moving Window Integration (150 ms)** | Haluskan puncak, lebarkan QRS |
| 5 | **Adaptive Thresholding** | Bedakan QRS dari noise secara adaptif |
| 6 | **Peak Detection** | Lokalisasi puncak QRS ke R-peak tepat |

Pada demo ini, kita implementasikan **langkah 1-4 secara manual** untuk memahami mekanismenya.

In [ ]:
# ============================================================
# Implementasi Manual Tahap 1-4 Pan-Tompkins
# ============================================================

# TAHAP 1: Bandpass filter 5-15 Hz
ecg_bp = bandpass_filter(ecg_proc, lowcut=5.0, highcut=15.0, fs=fs, order=4)

# TAHAP 2: Derivatif -- deteksi slope curam pada QRS
# Rumus: y[n] = (1/8) * (-x[n-2] - 2*x[n-1] + 2*x[n+1] + x[n+2])
# Ini adalah aproksimasi derivatif orde-1 dengan bobot (Pan & Tompkins, 1985)
ecg_deriv = np.zeros_like(ecg_bp)
for n in range(2, len(ecg_bp) - 2):
    ecg_deriv[n] = (1.0 / 8.0) * (
        -ecg_bp[n - 2] - 2 * ecg_bp[n - 1]
        + 2 * ecg_bp[n + 1] + ecg_bp[n + 2]
    )

# TAHAP 3: Kuadrasi -- perkuat QRS, semua nilai menjadi positif
ecg_sq = ecg_deriv ** 2

# TAHAP 4: Moving Window Integration (window = 150 ms)
window_mwi = int(0.150 * fs)  # 150 ms dalam sampel
ecg_mwi    = moving_average(ecg_sq, window_size=window_mwi)

print(f'Tahap Pan-Tompkins (manual):')
print(f'  [1] Bandpass 5-15 Hz    : shape={ecg_bp.shape}')
print(f'  [2] Derivatif           : shape={ecg_deriv.shape}')
print(f'  [3] Kuadrasi            : shape={ecg_sq.shape}')
print(f'  [4] MWI (window={window_mwi} sampel = 150 ms): shape={ecg_mwi.shape}')

# ---- Visualisasi seluruh tahap pada 4 detik pertama ----
t_pt   = 4.0  # tampilkan 4 detik
mask_pt = t <= t_pt

fig, axes = plt.subplots(5, 1, figsize=(14, 18), sharex=True)
fig.suptitle('Tahap-Tahap Algoritma Pan-Tompkins (Implementasi Manual)',
             fontsize=14, fontweight='bold')

signals_pt = [
    (ecg_proc,  'tab:blue',   '[0] ECG Terpreproses (input Pan-Tompkins)'),
    (ecg_bp,    'tab:cyan',   '[1] Setelah Bandpass 5-15 Hz'),
    (ecg_deriv, 'tab:green',  '[2] Setelah Derivatif (slope detector)'),
    (ecg_sq,    'tab:orange', '[3] Setelah Kuadrasi (semua positif, QRS diperkuat)'),
    (ecg_mwi,   'tab:red',    f'[4] Setelah MWI 150 ms (integrasi bergerak)'),
]

for i, (sig, color, title) in enumerate(signals_pt):
    axes[i].plot(t[mask_pt], sig[mask_pt], color=color, linewidth=1.0)
    axes[i].set_ylabel('Amplitudo')
    axes[i].set_title(title, fontsize=11)

axes[-1].set_xlabel('Waktu (s)')
plt.tight_layout()
plt.show()

# Perbandingan rasio QRS terhadap P/T
# Hitung amplitudo rata-rata pada waktu di sekitar R-peak vs di luarnya
r_mask_on  = np.zeros(len(t), dtype=bool)
r_mask_off = np.ones(len(t), dtype=bool)
win_qrs    = int(0.06 * fs)  # 60 ms di sekitar R-peak
for rp in r_peaks_gt:
    r_mask_on[max(0, rp - win_qrs):min(len(t), rp + win_qrs)] = True
r_mask_off = ~r_mask_on

ratio_raw = np.mean(ecg_proc[r_mask_on]**2) / (np.mean(ecg_proc[r_mask_off]**2) + 1e-9)
ratio_sq  = np.mean(ecg_sq[r_mask_on])      / (np.mean(ecg_sq[r_mask_off])      + 1e-9)
print(f'\nRasio daya QRS terhadap non-QRS:')
print(f'  Sinyal terpreproses : {ratio_raw:.1f}x')
print(f'  Setelah kuadrasi    : {ratio_sq:.1f}x')
print('Kuadrasi meningkatkan rasio QRS/non-QRS secara signifikan.')

### Mengapa Setiap Tahap Penting?

- **Bandpass 5-15 Hz**: Gelombang P dan T memiliki energi dominan di bawah 10 Hz; bandpass ini menekan P/T sekaligus mempertahankan QRS yang memiliki frekuensi dominan 10-20 Hz.
- **Derivatif**: QRS memiliki *slope* (kemiringan) yang sangat curam dibanding P dan T. Derivatif mengubah kemiringan menjadi amplitudo -- QRS menjadi paling menonjol.
- **Kuadrasi**: (1) Membuat semua nilai positif sehingga tidak ada penjumlahan yang saling menghapus; (2) Secara non-linear memperkuat puncak besar (QRS) lebih banyak dari puncak kecil (noise).
- **Moving Window Integration**: Menghaluskan sinyal kuadrat menjadi "puncak lebar" yang mudah dideteksi, sekaligus menggabungkan energi dari beberapa titik dalam satu QRS.

---
## 5. Deteksi R-Peak dengan medsinyal

Setelah memahami prinsip Pan-Tompkins secara manual, kita gunakan implementasi yang sudah dioptimalkan dari library `medsinyal`. Kemudian kita evaluasi kinerjanya dengan membandingkan terhadap **ground truth** (*R-peak indices* dari data sintetis).

### Metrik Evaluasi Detektor R-peak

| Metrik | Rumus | Keterangan |
|--------|-------|------------|
| **True Positive (TP)** | Peak terdeteksi yang sesuai dengan ground truth | |
| **False Positive (FP)** | Peak terdeteksi yang **tidak** ada di ground truth | |
| **False Negative (FN)** | Peak di ground truth yang **tidak** terdeteksi | |
| **Sensitivity (Se)** | $\frac{TP}{TP+FN}$ | Kemampuan mendeteksi peak yang benar |
| **PPV** (*Positive Predictive Value*) | $\frac{TP}{TP+FP}$ | Presisi -- seberapa banyak deteksi yang benar |

In [ ]:
# Deteksi R-peak dengan medsinyal (Pan-Tompkins yang dioptimalkan)
r_peaks_det = detect_r_peaks(
    ecg_proc, fs,
    bandpass_range=(5.0, 15.0),
    threshold_factor=0.6,
    min_distance_s=0.2
)

print(f'Jumlah R-peak ground truth : {len(r_peaks_gt)}')
print(f'Jumlah R-peak terdeteksi   : {len(r_peaks_det)}')

# ---- Evaluasi: hitung TP, FP, FN ----
# Toleransi: dua peak dianggap cocok jika selisihnya < 75 ms
tolerance = int(0.075 * fs)

tp_list = []
matched_gt = set()
matched_det = set()

for i_det, rp_det in enumerate(r_peaks_det):
    for i_gt, rp_gt in enumerate(r_peaks_gt):
        if abs(int(rp_det) - int(rp_gt)) <= tolerance and i_gt not in matched_gt:
            tp_list.append((rp_det, rp_gt))
            matched_gt.add(i_gt)
            matched_det.add(i_det)
            break

TP = len(tp_list)
FP = len(r_peaks_det) - len(matched_det)
FN = len(r_peaks_gt)  - len(matched_gt)

sensitivity = TP / (TP + FN) if (TP + FN) > 0 else 0.0
ppv         = TP / (TP + FP) if (TP + FP) > 0 else 0.0

print(f'\nEvaluasi Deteksi R-peak (toleransi = {tolerance} sampel = 75 ms):')
print(f'  True Positive  (TP): {TP}')
print(f'  False Positive (FP): {FP}')
print(f'  False Negative (FN): {FN}')
print(f'  Sensitivity (Se)   : {sensitivity*100:.2f}%')
print(f'  PPV                 : {ppv*100:.2f}%')

# Visualisasi deteksi vs ground truth
fig, axes = plt.subplots(2, 1, figsize=(14, 8), sharex=True)
fig.suptitle('Deteksi R-peak: Ground Truth vs Hasil Deteksi', fontsize=14, fontweight='bold')

plot_ecg(t, ecg_proc, r_peaks=r_peaks_gt,
         title='Ground Truth R-peak (segitiga hijau)', ax=axes[0])
axes[0].set_ylabel('Amplitudo (mV)')

plot_ecg(t, ecg_proc, r_peaks=r_peaks_det,
         title=f'R-peak Terdeteksi (Se={sensitivity*100:.1f}%, PPV={ppv*100:.1f}%)',
         ax=axes[1])
axes[1].set_ylabel('Amplitudo (mV)')

plt.tight_layout()
plt.show()

---
## 6. Interval RR dan Denyut Jantung (*RR Intervals & Heart Rate*)

**Interval RR** (*RR interval*) adalah jarak waktu antara dua R-peak yang berurutan. Ini adalah representasi langsung dari **siklus detak jantung** (*cardiac cycle*).

Dari interval RR, kita dapat menghitung:
- **Denyut jantung sesaat** (*instantaneous heart rate*): $\text{HR}[n] = 60 / \text{RR}[n]$ bpm
- **Variabilitas denyut jantung** (*Heart Rate Variability / HRV*): fluktuasi interval RR dari waktu ke waktu

**Takogram RR** (*RR tachogram*) adalah plot interval RR sebagai fungsi nomor detak -- alat dasar untuk analisis HRV.

In [ ]:
# Hitung interval RR dan heart rate dari R-peak terdeteksi
rr_intervals = compute_rr_intervals(r_peaks_det, fs)   # dalam detik
hr_instant   = compute_heart_rate(rr_intervals)         # dalam bpm

# Waktu tengah setiap interval RR (rata-rata dua R-peak berurutan)
t_rr = t[r_peaks_det[:-1]] + rr_intervals / 2.0
beat_numbers = np.arange(1, len(rr_intervals) + 1)

print(f'Interval RR (detik):')
print(f'  Mean : {np.mean(rr_intervals):.4f} s  ({60/np.mean(rr_intervals):.1f} bpm)')
print(f'  Std  : {np.std(rr_intervals):.4f} s')
print(f'  Min  : {np.min(rr_intervals):.4f} s')
print(f'  Max  : {np.max(rr_intervals):.4f} s')

fig, axes = plt.subplots(3, 1, figsize=(14, 12))
fig.suptitle('Interval RR dan Denyut Jantung Sesaat (*Instantaneous Heart Rate*)',
             fontsize=14, fontweight='bold')

# --- Panel 1: ECG dengan R-peak ---
plot_ecg(t, ecg_proc, r_peaks=r_peaks_det, ax=axes[0],
         title='ECG dengan R-peak Terdeteksi')
axes[0].set_ylabel('Amplitudo (mV)')

# --- Panel 2: Takogram RR (RR vs nomor detak) ---
axes[1].stem(beat_numbers, rr_intervals * 1000,
             linefmt='tab:blue', markerfmt='bo', basefmt='gray')
axes[1].axhline(y=np.mean(rr_intervals) * 1000, color='red',
                linestyle='--', linewidth=1.5,
                label=f'Mean RR = {np.mean(rr_intervals)*1000:.1f} ms')
axes[1].set_xlabel('Nomor Detak (*Beat Number*)')
axes[1].set_ylabel('Interval RR (ms)')
axes[1].set_title('Takogram RR (*RR Tachogram*) -- Dasar Analisis HRV')
axes[1].legend(loc='upper right')

# --- Panel 3: Denyut jantung sesaat vs waktu ---
axes[2].plot(t_rr, hr_instant, 'o-', color='tab:orange',
             markersize=5, linewidth=1.5)
axes[2].axhline(y=np.mean(hr_instant), color='red',
                linestyle='--', linewidth=1.5,
                label=f'Mean HR = {np.mean(hr_instant):.1f} bpm')
axes[2].set_xlabel('Waktu (s)')
axes[2].set_ylabel('Heart Rate (bpm)')
axes[2].set_title('Denyut Jantung Sesaat (*Instantaneous Heart Rate*)')
axes[2].legend(loc='upper right')

plt.tight_layout()
plt.show()

print(f'\nGround-truth heart rate: {hr_gt:.1f} bpm')
print(f'Mean HR dari deteksi   : {np.mean(hr_instant):.1f} bpm')
print(f'Selisih                : {abs(np.mean(hr_instant) - hr_gt):.2f} bpm')

---
## 7. Analisis HRV (*Heart Rate Variability Analysis*)

**HRV** (*Heart Rate Variability*) mengukur fluktuasi interval RR dari waktu ke waktu. HRV merupakan indikator aktivitas **sistem saraf otonom** (*autonomic nervous system / ANS*):

- **Sistem saraf simpatis** (*sympathetic*): respons "fight or flight", mempercepat jantung, menurunkan HRV
- **Sistem saraf parasimpatis** (*parasympathetic / vagal*): respons "rest and digest", memperlambat jantung, meningkatkan HRV

### Fitur HRV Domain Waktu (*Time-Domain HRV Features*)

| Fitur | Rumus | Interpretasi |
|-------|-------|-------------|
| **mean_rr** | $\overline{RR}$ | Durasi siklus kardiak rata-rata |
| **SDNN** | $\sqrt{\text{Var}(RR)}$ | HRV keseluruhan; aktivitas total ANS |
| **RMSSD** | $\sqrt{\frac{1}{N-1}\sum_{i=1}^{N-1}(RR_{i+1}-RR_i)^2}$ | HRV jangka pendek; tonus parasimpatis |
| **pNN50** | $\frac{\text{jumlah } |\Delta RR| > 50\text{ ms}}{N-1} \times 100\%$ | Persentase perbedaan RR berurutan > 50 ms |
| **mean_hr** | $60 / \overline{RR}$ | Denyut jantung rata-rata (bpm) |
| **std_hr** | $\text{std}(\text{HR sesaat})$ | Variabilitas denyut jantung (bpm) |

### Interpretasi Klinis

| Kondisi | SDNN | RMSSD | Interpretasi |
|---------|------|-------|-------------|
| Individu sehat, istirahat | > 50 ms | > 30 ms | Tonus vagal baik |
| Stres akut | Menurun | Menurun | Dominasi simpatis |
| Pasca-infark miokard | < 20 ms | < 15 ms | Disfungsi otonom; risiko tinggi |
| Atlet terlatih | Tinggi | Tinggi | Tonus vagal sangat baik |
| Diabetes neuropati | Sangat rendah | Sangat rendah | Kerusakan ANS |

In [ ]:
# Hitung fitur HRV domain waktu menggunakan medsinyal
hrv_features = compute_hrv_features(rr_intervals)

print('=' * 55)
print('HASIL ANALISIS HRV -- DOMAIN WAKTU')
print('=' * 55)

hrv_display = [
    ('mean_rr',  'Mean RR',   'ms',  hrv_features['mean_rr']  * 1000),
    ('sdnn',     'SDNN',      'ms',  hrv_features['sdnn']     * 1000),
    ('rmssd',    'RMSSD',     'ms',  hrv_features['rmssd']    * 1000),
    ('pnn50',    'pNN50',     '%',   hrv_features['pnn50']),
    ('mean_hr',  'Mean HR',   'bpm', hrv_features['mean_hr']),
    ('std_hr',   'Std HR',    'bpm', hrv_features['std_hr']),
]

print(f'  {"Fitur":<25} {"Nilai":>10} {"Satuan":<6}')
print('  ' + '-' * 45)
for key, label, unit, val in hrv_display:
    print(f'  {label:<25} {val:>10.3f} {unit:<6}')

print('\nInterpretasi:')
sdnn_ms  = hrv_features['sdnn']  * 1000
rmssd_ms = hrv_features['rmssd'] * 1000
pnn50    = hrv_features['pnn50']

if sdnn_ms > 50:
    print(f'  SDNN = {sdnn_ms:.1f} ms  --> HRV baik; tonus otonom sehat.')
elif sdnn_ms > 20:
    print(f'  SDNN = {sdnn_ms:.1f} ms  --> HRV sedang; perlu evaluasi lebih lanjut.')
else:
    print(f'  SDNN = {sdnn_ms:.1f} ms  --> HRV rendah; risiko disfungsi otonom.')

if rmssd_ms > 30:
    print(f'  RMSSD = {rmssd_ms:.1f} ms --> Aktivitas parasimpatis (vagal) baik.')
else:
    print(f'  RMSSD = {rmssd_ms:.1f} ms --> Aktivitas parasimpatis rendah; dominasi simpatis.')

In [ ]:
# --- Visualisasi komprehensif HRV ---
fig, axes = plt.subplots(2, 2, figsize=(14, 10))
fig.suptitle('Analisis HRV Domain Waktu (*Time-Domain HRV Analysis*)',
             fontsize=14, fontweight='bold')

rr_ms = rr_intervals * 1000  # konversi ke ms

# Panel (0,0): Takogram RR
axes[0, 0].plot(beat_numbers, rr_ms, 'b-o', markersize=5, linewidth=1.5)
axes[0, 0].axhline(y=np.mean(rr_ms), color='red', linestyle='--',
                   label=f'Mean = {np.mean(rr_ms):.1f} ms')
axes[0, 0].fill_between(
    beat_numbers,
    np.mean(rr_ms) - hrv_features['sdnn'] * 1000,
    np.mean(rr_ms) + hrv_features['sdnn'] * 1000,
    alpha=0.2, color='red', label=f'± SDNN ({hrv_features["sdnn"]*1000:.1f} ms)'
)
axes[0, 0].set_xlabel('Nomor Detak')
axes[0, 0].set_ylabel('Interval RR (ms)')
axes[0, 0].set_title('Takogram RR')
axes[0, 0].legend(fontsize=9)

# Panel (0,1): Histogram distribusi RR
axes[0, 1].hist(rr_ms, bins=20, color='tab:blue', edgecolor='white',
                alpha=0.8, density=True)
axes[0, 1].axvline(x=np.mean(rr_ms), color='red', linestyle='--',
                   linewidth=2, label=f'Mean = {np.mean(rr_ms):.1f} ms')
axes[0, 1].set_xlabel('Interval RR (ms)')
axes[0, 1].set_ylabel('Densitas')
axes[0, 1].set_title('Distribusi Interval RR')
axes[0, 1].legend(fontsize=9)

# Panel (1,0): Perbedaan RR berurutan (ΔRR) -- dasar RMSSD dan pNN50
delta_rr = np.diff(rr_ms)
axes[1, 0].bar(beat_numbers[:-1], np.abs(delta_rr), color='tab:orange',
               alpha=0.8, edgecolor='white')
axes[1, 0].axhline(y=50, color='red', linestyle='--', linewidth=1.5,
                   label='Ambang pNN50 = 50 ms')
axes[1, 0].set_xlabel('Nomor Detak')
axes[1, 0].set_ylabel('|ΔRR| (ms)')
axes[1, 0].set_title(f'Perbedaan RR Berurutan |ΔRR| -- pNN50 = {pnn50:.1f}%')
axes[1, 0].legend(fontsize=9)

# Panel (1,1): Poincare plot (scatter RR[n] vs RR[n+1])
axes[1, 1].scatter(rr_ms[:-1], rr_ms[1:], color='tab:purple',
                   alpha=0.7, s=40, edgecolors='white', linewidths=0.5)
# Garis identitas (RR[n] = RR[n+1])
rr_lim = [rr_ms.min() - 5, rr_ms.max() + 5]
axes[1, 1].plot(rr_lim, rr_lim, 'k--', linewidth=1, alpha=0.5, label='y = x')
axes[1, 1].set_xlabel('RR[n] (ms)')
axes[1, 1].set_ylabel('RR[n+1] (ms)')
axes[1, 1].set_title('Poincaré Plot (*Poincaré Plot*) -- Nonlinear HRV')
axes[1, 1].legend(fontsize=9)

plt.tight_layout()
plt.show()

print('Poincaré plot: Penyebaran titik di sepanjang garis y=x menunjukkan variabilitas jangka panjang.')
print('Penyebaran tegak lurus garis y=x menunjukkan variabilitas jangka pendek (RMSSD).')

---
## 8. Kesimpulan (*Conclusion*)

### Ringkasan Materi Minggu 5

Pada pertemuan ini kita telah mempelajari:

1. **Sistem konduksi jantung** menghasilkan sinyal ECG melalui jalur SA Node $\rightarrow$ AV Node $\rightarrow$ Bundle of His $\rightarrow$ Serat Purkinje. Gangguan pada setiap bagian jalur ini menghasilkan pola ECG abnormal yang dapat didiagnosis secara klinis.

2. **Kompleks PQRST** memiliki nilai normal yang baku. Perubahan durasi atau amplitudo komponen ini mengindikasikan kondisi seperti blok AV, bundle branch block, atau long QT syndrome.

3. **Preprocessing ECG** dilakukan secara bertahap: notch filter 50 Hz untuk menghilangkan interferensi jala listrik, baseline wander removal untuk menghilangkan drift frekuensi rendah, dan bandpass filter 0.5-40 Hz untuk mempertahankan komponen ECG yang relevan.

4. **Algoritma Pan-Tompkins** meningkatkan detektabilitas QRS melalui empat tahap berurutan: bandpass 5-15 Hz, derivatif (memperkuat kemiringan), kuadrasi (memperkuat QRS non-linear), dan moving window integration (menghaluskan hasil).

5. **Evaluasi detektor R-peak** menggunakan Sensitivity dan PPV -- dua metrik yang menggambarkan kemampuan detektor secara komplementer. Detektor yang baik memiliki keduanya mendekati 100%.

6. **HRV** (*Heart Rate Variability*) adalah ukuran fluktuasi interval RR yang merefleksikan keseimbangan sistem saraf otonom. SDNN mengukur HRV keseluruhan, RMSSD dan pNN50 mengukur aktivitas parasimpatis jangka pendek. HRV rendah berkaitan dengan stres, patologi kardiovaskular, dan peningkatan risiko kardiak.

### Minggu Depan

Kita akan membahas **Fitur ECG & Klasifikasi Aritmia** (*ECG Features & Arrhythmia Classification*) -- bagaimana mengekstraksi fitur morfologis dari kompleks PQRST dan menggunakan *machine learning* untuk klasifikasi otomatis jenis aritmia.

---
## Referensi

1. **Pan, J. & Tompkins, W.J.** (1985). *A real-time QRS detection algorithm*. IEEE Transactions on Biomedical Engineering, 32(3), 230-236. DOI: 10.1109/TBME.1985.325532

2. **Rangayyan, R.M.** (2015). *Biomedical Signal Analysis* (2nd ed.). Wiley-IEEE Press. -- Chapters 6-8 (ECG Analysis, Filtering, Feature Extraction)

3. **Sörnmo, L. & Laguna, P.** (2005). *Bioelectrical Signal Processing in Cardiac and Neurological Applications*. Academic Press. -- Chapters 5-7 (ECG Modeling, QRS Detection, HRV)

4. **Task Force of the European Society of Cardiology.** (1996). *Heart rate variability: Standards of measurement, physiological interpretation, and clinical use*. Circulation, 93(5), 1043-1065.

5. **Clifford, G.D., Azuaje, F., & McSharry, P.** (2006). *Advanced Methods and Tools for ECG Data Analysis*. Artech House. -- Open access melalui PhysioNet.